In [ ]:
import cPickle
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from IPython import display
import scipy.sparse as sp
from scipy.stats import pearsonr
import os
os.chdir('/Users/zbutler/research/fire_prediction')
from geometry.get_xys import append_xy
from prediction.fire_clustering import cluster_fires
from geometry.grid_conversion import ak_bb
from plotting.plot_map import draw_map_nogrid_static
from math import floor
%matplotlib inline

In [ ]:
# First, load MODIS active fire detection data
# Load in our pretty pandas database
with open("data/modis.pkl") as fpd:
    modis = cPickle.load(fpd)
print "loaded pandas database with %d points" %len(modis)
print "min year: %d, max year: %d" %(np.min(modis.year), np.max(modis.year))

In [ ]:
# Fires per year
total_fires_arr = []
month_float_arr = []
year = np.min(modis.year)
month = 1
while 1:
    n_fires = len(modis[(modis.year==year) & (modis.month==month)])
    if not n_fires:
        break
    total_fires_arr.append(n_fires)
    month_float_arr.append(year + (float(month-1) / 12.))
    month += 1
    if month == 13:
        year += 1
        month = 1

plt.plot(month_float_arr, total_fires_arr, 'r')
plt.title("Total fires per month")
plt.show()

In [ ]:
# Now fires for Alaska
ak_fires = modis[(modis.lat < ak_bb[1]) & (modis.lat > ak_bb[0]) & (modis.long > ak_bb[2]) & (modis.long < ak_bb[3])]
ak_fires_arr = []
year = np.min(modis.year)
month = 1
last_month = int(month_float_arr[-1] * 12) % 12
last_year = int(floor(month_float_arr[-1]))
while year < last_year or month <= last_month+1:
    n_fires = len(ak_fires[(ak_fires.year==year) & (ak_fires.month==month)])
    ak_fires_arr.append(n_fires)
    month += 1
    if month == 13:
        year += 1
        month = 1

plt.plot(month_float_arr, ak_fires_arr, 'r')
plt.show()

In [ ]:
ak_summer_fires = ak_fires[(ak_fires.year == 2013) & (ak_fires.month >= 6) & (ak_fires.month <= 8)]

plt.figure(figsize=(7,14))
mp = Basemap(projection="merc",
              lat_0=ak_bb[0], lon_0=ak_bb[2],
              llcrnrlat=ak_bb[0],
              llcrnrlon=ak_bb[2],
              urcrnrlat=ak_bb[1],
              urcrnrlon=ak_bb[3],
              resolution='i')
month = 6
day = 1
prev_lats = []
prev_longs = []
while month < 9:
    todays_fires = ak_fires[(ak_fires.year == 2013) & (ak_fires.month == month) &
                            (ak_fires.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(7,14))
    mp.drawmapboundary(fill_color='aqua')
    mp.fillcontinents(color='yellow',lake_color='aqua')
    mp.drawcoastlines()
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(todays_fires):
        today_longs, today_lats = mp(np.array(todays_fires.long), np.array(todays_fires.lat))
        mp.plot(today_longs, today_lats, 'ro')
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    plt.title("Fires in Alaska on %d %d" %(month, day))
    plt.show()
    time.sleep(.8)
    day += 1
    if day >= 31 or (day >= 30 and month == 6):
        day = 1
        month += 1

In [ ]:
plt.figure(figsize=(8,8))
fire_2_bb = [56.15, 56.25, -159.5, -159.3]
mp = Basemap(projection="merc",
              lat_0=float(fire_2_bb[0]+fire_2_bb[1])/2, lon_0=float(fire_2_bb[2]+fire_2_bb[3])/2,
              llcrnrlat=fire_2_bb[0],
              llcrnrlon=fire_2_bb[2],
              urcrnrlat=fire_2_bb[1],
              urcrnrlon=fire_2_bb[3])
my_fires_2 = modis[(modis.lat < fire_2_bb[1]) & (modis.lat > fire_2_bb[0]) &
                   (modis.long < fire_2_bb[3]) & (modis.long > fire_2_bb[2])]
month = 6
day = 14
prev_lats = []
prev_longs = []
while month < 10 or day <= 11:
    todays_fires = my_fires_2[(my_fires_2.year == 2013) & (my_fires_2.month == month) &
                            (my_fires_2.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(5,10))
    mp.drawmapboundary(fill_color='aqua')
    mp.fillcontinents(color='yellow',lake_color='aqua')
    mp.drawcoastlines()
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(todays_fires):
        today_longs, today_lats = mp(np.array(todays_fires.long), np.array(todays_fires.lat))
        mp.plot(today_longs, today_lats, 'ro')
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    plt.title("Fires in 5km BB on %d %d" %(month, day))
    plt.show()
    time.sleep(1)
    day += 1
    if day >= 31 or (day >= 30 and month == 6):
        day = 1
        month += 1
        
my_2013_fires_2 = my_fires_2[my_fires_2.year==2013]
print "%d fires in this region %d in 2013" % (len(my_fires_2), len(my_2013_fires_2))
earliest_month = min(my_2013_fires_2.month)
latest_month = max(my_2013_fires_2.month)
earliest_day = min(my_2013_fires_2[my_2013_fires_2.month == earliest_month].day)
latest_day = max(my_2013_fires_2[my_2013_fires_2.month == latest_month].day)
print "fires in this region in 2013 range from %d/%d-%d/%d" \
        %(earliest_month,earliest_day, latest_month, latest_day)

In [ ]:
with open("data/temp_dict.pkl") as fpkl:
    temp_dict = cPickle.load(fpkl)
with open("data/hum_dict.pkl") as fpkl:
    hum_dict = cPickle.load(fpkl)
with open("data/vpd_dict.pkl") as fpkl:
    vpd_dict = cPickle.load(fpkl)

In [ ]:
plt.figure(figsize=(7,14))
draw_map_nogrid_static(ak_bb, month=1, day=15, gfs_dict=temp_dict)
plt.figure(figsize=(7,14))
draw_map_nogrid_static(ak_bb, month=1, day=15, gfs_dict=hum_dict)
plt.figure(figsize=(7,14))
draw_map_nogrid_static(ak_bb, month=1, day=15, gfs_dict=vpd_dict)

In [ ]:
ak_fires = append_xy(ak_fires, ak_bb)

In [ ]:
threshes = [1,5,10,20,50]
clusts_arr = []
n_clusts_arr = []
ak_2013_fires = ak_fires[ak_fires.year == 2015]
#for thresh in threshes:
#    n_clusts, clusts = cluster_fires(ak_2013_fires, thresh)
#    clusts_arr.append(clusts)
#    n_clusts_arr.append(n_clusts)
#    print "done with thresh %d" %(thresh)
#plt.plot(threshes, n_clusts_arr, 'rs')

In [ ]:
import metrics.fire_metrics as fm
#n_clusts, clusts = cluster_fires(ak_2013_fires, 20)
#print "%d fires" %(n_clusts)
summary_stat_dict = fm.get_summary_stats(ak_2013_fires, clusts, n_clusts)
time_series = fm.get_time_series(ak_2013_fires, clusts, n_clusts, zero_centered=False)
time_series_0_centered = fm.get_time_series(ak_2013_fires, clusts, n_clusts, zero_centered=True)
# Plots!
plt.hist(summary_stat_dict['len'], bins=100)
plt.title("histogram of fire sizes")
plt.show()

plt.hist(np.log(summary_stat_dict['len'] + 1), bins=100)
plt.title("Log histogram of fire sizes")
plt.show()

plt.hist(summary_stat_dict['area'], bins=100)  # convert to km
plt.title("Histogram of convex hull of fires")
plt.show()

plt.hist(summary_stat_dict['dist_from_center'], bins=100) # convert to km
plt.title("Histogram of mean distance from cluster center (in km)")
plt.show()

plt.hist(summary_stat_dict['duration'], bins=100)
plt.title("Histograms of fire duration")
plt.show()

decay_arr = np.zeros(int(np.max(summary_stat_dict['duration'])))
for ln in xrange(int(np.max(summary_stat_dict['duration']))):
    n_big = np.sum(summary_stat_dict['duration'] >= ln)
    decay_arr[ln] = n_big
plt.plot(decay_arr,'b')
plt.show()

In [ ]:
# Time series
lat_arr = np.zeros(n_clusts)
for clust in xrange(n_clusts):
    center_lat = np.mean(ak_2013_fires[clusts == clust].long)
    lat_arr[clust] = center_lat
sorted_lats = np.argsort(lat_arr)
for clust,series in enumerate(time_series):
    plt.plot(series, np.zeros(len(series)) + sorted_lats[clust], 'r.')
plt.title("Time series (not zero centered)")
plt.show()
plt.close()

for clust,series in enumerate(time_series_0_centered):
    plt.plot(series, np.zeros(len(series)) + sorted_lats[clust], 'r.')
plt.title("Time series (zero centered)")
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(7,14))
centers = np.zeros((n_clusts,2))
for fire in xrange(n_clusts):
    clust_fires = ak_2013_fires[clusts == fire]
    centers[fire,0] = np.mean(clust_fires.lat)
    centers[fire,1] = np.mean(clust_fires.long)

mp = Basemap(projection="merc",
                 lat_0=ak_bb[0], lon_0=ak_bb[2],
                 llcrnrlat=ak_bb[0],
                 llcrnrlon=ak_bb[2],
                 urcrnrlat=ak_bb[1],
                 urcrnrlon=ak_bb[3],
                 resolution='i')
center_lons, center_lats = mp(centers[:,1], centers[:,0])
mp.plot(center_lons, center_lats, 'rs')
mp.drawmapboundary(fill_color='aqua')
mp.fillcontinents(color='yellow',lake_color='aqua')
mp.drawcoastlines()
plt.show()

In [ ]:
# load in feat df
with open('data/feat_dfs/feat_df_2013.pkl') as fpkl:
    feat_df = cPickle.load(fpkl)
feat_df

In [ ]:
print "VPD correlation: " + str(pearsonr(feat_df.n_det, feat_df.vpd))
print "Humidity correlation: " + str(pearsonr(feat_df.n_det, feat_df.humidity))
print "Temperature correlation: " + str(pearsonr(feat_df.n_det, feat_df.temp))

In [ ]:
df_arr = []
for year in xrange(2010,2017):
    with open('data/feat_dfs/feat_df_%d.pkl' %year) as fpkl:
        df = cPickle.load(fpkl)
        df['year'] = pd.Series(np.zeros(len(df)) + year, index=df.index)
        df_arr.append(df)
        
big_df = pd.concat(df_arr)
df_arr[0][df_arr[0].fire_id == 1]

In [ ]:
print "VPD correlation: " + str(pearsonr(feat_df.n_det, feat_df.vpd))
print "Humidity correlation: " + str(pearsonr(feat_df.n_det, feat_df.humidity))
print "Temperature correlation: " + str(pearsonr(feat_df.n_det, feat_df.temp))

In [ ]:
clust_id = 0
clusts = []
for year in xrange(2010, 2017):
    clusts = clusts + list(big_df.fire_id + clust_id)
    clust_id += len(big_df.fire_id)
n_clusts = len(np.unique(clusts))

In [ ]:
summary_stat_dict = fm.get_summary_stats(big_df, clusts, n_clusts)
time_series = fm.get_time_series(ak_2013_fires, clusts, n_clusts, zero_centered=False)
time_series_0_centered = fm.get_time_series(ak_2013_fires, clusts, n_clusts, zero_centered=True)
# Plots!
plt.hist(summary_stat_dict['len'], bins=100)
plt.title("histogram of fire sizes")
plt.show()

plt.hist(np.log(summary_stat_dict['len'] + 1), bins=100)
plt.title("Log histogram of fire sizes")
plt.show()

plt.hist(summary_stat_dict['area'], bins=100)  # convert to km
plt.title("Histogram of convex hull of fires")
plt.show()

plt.hist(summary_stat_dict['dist_from_center'], bins=100) # convert to km
plt.title("Histogram of mean distance from cluster center (in km)")
plt.show()

plt.hist(summary_stat_dict['duration'], bins=100)
plt.title("Histograms of fire duration")
plt.show()


In [ ]:
temp_vals = 20
print .6108 * np.exp(17.27 * temp_vals / (temp_vals + 237.3))